In [1]:
import os
import numpy as np
import pandas as pd
import time
from selenium import webdriver
import csv
from selenium.webdriver.common.keys import Keys

In [5]:
df = pd.read_csv("financial_data/financial_data_200001_202108_monthly.csv",encoding="SHIFT-JIS")
df

,期間,日経平均,TOPIX,東証2部総合,東証マザーズ,JASDAQ,株式コード,上場コード,銘柄名称,株式区分,...,日経業種小分類コード,決算月（年２回決算）,決算月,発行済み株式数（権利落ベース）,発行済み株式数（権利落ベース）変更日,日経平均採用区分,日経３００採用区分,日経５００採用区分,東証：上場日,東証：上場廃止日
0,2000/01,19539.70,1707.96,2669.39,NaN,103.81,1717,NaN,明豊ファシリティワークス,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000/02,19959.52,1718.94,3070.73,NaN,125.73,1717,NaN,明豊ファシリティワークス,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000/03,20337.32,1705.94,2809.67,NaN,108.94,1717,NaN,明豊ファシリティワークス,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000/04,17973.70,1648.87,2645.03,NaN,100.17,1717,NaN,明豊ファシリティワークス,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000/05,16332.45,1522.84,2440.13,NaN,85.71,1717,NaN,明豊ファシリティワークス,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379075,2021/04,28812.63,1898.24,7453.80,1201.77,185.37,9991,11.0,ジェコス,,...,704.0,0.0,3.0,36436125.0,19970326.0,NaN,NaN,NaN,19940826.0,NaN
379076,2021/05,28860.08,1922.98,7458.27,1150.06,182.74,9991,11.0,ジェコス,,...,704.0,0.0,3.0,36436125.0,19970326.0,NaN,NaN,NaN,19940826.0,NaN
379077,2021/06,28791.53,1943.57,7675.60,1207.46,187.63,9991,11.0,ジェコス,,...,704.0,0.0,3.0,36436125.0,19970326.0,NaN,NaN,NaN,19940826.0,NaN
379078,2021/07,27283.59,1901.08,7678.15,1085.34,185.39,9991,11.0,ジェコス,,...,704.0,0.0,3.0,36436125.0,19970326.0,NaN,NaN,NaN,19940826.0,NaN


In [12]:
#基本統計量
df.describe()

,日経平均,TOPIX,東証2部総合,東証マザーズ,JASDAQ,株式コード,上場コード,取引所,上場場部,始値（権利落調整済）,...,日経業種小分類コード,決算月（年２回決算）,決算月,発行済み株式数（権利落ベース）,発行済み株式数（権利落ベース）変更日,日経平均採用区分,日経３００採用区分,日経５００採用区分,東証：上場日,東証：上場廃止日
count,377622.000000,377622.000000,377622.000000,313470.000000,377622.000000,379080.000000,185153.00000,185153.000000,185153.00000,1.840430e+05,...,185153.000000,185153.0,185153.000000,1.851530e+05,1.851530e+05,1503.0,5385.0,8006.0,1.133480e+05,3.170000e+02
mean,15043.020618,1290.130811,3780.996911,948.831814,93.712934,5211.056927,43.14158,3.224474,43.14158,1.784685e+04,...,695.161596,0.0,5.315750,2.652282e+07,2.008318e+07,1.0,1.0,1.0,2.008127e+07,2.013124e+07
std,5160.597806,337.366216,1808.325092,479.682542,42.892595,2364.872235,39.23404,3.663250,39.23404,1.228584e+05,...,55.982147,0.0,3.527635,1.788170e+08,7.314622e+04,0.0,0.0,0.0,6.120189e+04,4.991015e+04
min,7568.420000,719.490000,1485.100000,297.210000,38.490000,1717.000000,11.00000,1.000000,11.00000,1.000000e+00,...,10.000000,0.0,1.000000,3.200000e+03,1.976092e+07,1.0,1.0,1.0,1.994083e+07,2.004032e+07
25%,10395.180000,975.660000,2210.390000,510.060000,52.110000,3690.000000,11.00000,1.000000,11.00000,4.400000e+02,...,704.000000,0.0,3.000000,2.106900e+06,2.003080e+07,1.0,1.0,1.0,2.003090e+07,2.009072e+07
50%,14455.800000,1300.980000,3329.100000,907.560000,89.290000,4440.500000,14.00000,1.000000,14.00000,8.940000e+02,...,704.000000,0.0,3.000000,7.690000e+06,2.008023e+07,1.0,1.0,1.0,2.008012e+07,2.013073e+07
75%,19033.710000,1591.600000,4842.050000,1154.520000,116.950000,7033.000000,90.00000,8.000000,90.00000,1.965000e+03,...,704.000000,0.0,9.000000,1.973889e+07,2.014103e+07,1.0,1.0,1.0,2.013072e+07,2.017033e+07
max,29178.800000,1954.000000,7678.150000,2672.310000,189.080000,9991.000000,112.00000,11.000000,112.00000,1.250000e+07,...,704.000000,0.0,12.000000,1.497090e+10,2.021073e+07,1.0,1.0,1.0,2.021073e+07,2.021082e+07


In [9]:
#ユニークな値の要素の個数
df.nunique()

期間                       260
日経平均                     259
TOPIX                    259
東証2部総合                   259
東証マザーズ                   215
JASDAQ                   252
株式コード                   1458
上場コード                     20
銘柄名称                    1452
株式区分                       1
取引所                        8
上場場部                      20
始値（権利落調整済）              8514
高値（権利落調整済）              8749
安値（権利落調整済）              8154
終値（権利落調整済）              8663
時価総額（発行済み株式数ベース）      163382
時価総額（普通株式数ベース）        146994
東証業種コード３３分類               16
日経業種区分                     2
日経業種中分類コード                13
日経業種小分類コード                21
決算月（年２回決算）                 1
決算月                       12
発行済み株式数（権利落ベース）        24284
発行済み株式数（権利落ベース）変更日      2875
日経平均採用区分                   1
日経３００採用区分                  1
日経５００採用区分                  1
東証：上場日                   775
東証：上場廃止日                 174
dtype: int64

In [10]:
#最頻値
df.mode()

,期間,日経平均,TOPIX,東証2部総合,東証マザーズ,JASDAQ,株式コード,上場コード,銘柄名称,株式区分,...,日経業種小分類コード,決算月（年２回決算）,決算月,発行済み株式数（権利落ベース）,発行済み株式数（権利落ベース）変更日,日経平均採用区分,日経３００採用区分,日経５００採用区分,東証：上場日,東証：上場廃止日
0,2000/01,7568.42,719.49,1485.10,297.21,47.87,1717,11.0,オークネット,,...,704.0,0.0,3.0,3500000.0,20130926.0,1.0,1.0,1.0,20130716.0,20060926.0
1,2000/02,7831.42,728.46,1499.02,299.24,51.26,1901,NaN,チムニー,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20100126.0
2,2000/03,7972.71,728.61,1512.55,306.45,51.89,1973,NaN,マクロミル,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000/04,7994.05,731.64,1519.40,308.84,52.37,2120,NaN,ラック,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000/05,8109.53,736.31,1553.31,313.76,75.03,2121,NaN,日本アジアグループ,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1453,NaN,NaN,NaN,NaN,NaN,NaN,9961,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1454,NaN,NaN,NaN,NaN,NaN,NaN,9969,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1455,NaN,NaN,NaN,NaN,NaN,NaN,9979,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1456,NaN,NaN,NaN,NaN,NaN,NaN,9985,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
#全ての要素が欠損値となる列が存在するか
df.isnull().all()

期間                    False
日経平均                  False
TOPIX                 False
東証2部総合                False
東証マザーズ                False
JASDAQ                False
株式コード                 False
上場コード                 False
銘柄名称                  False
株式区分                  False
取引所                   False
上場場部                  False
始値（権利落調整済）            False
高値（権利落調整済）            False
安値（権利落調整済）            False
終値（権利落調整済）            False
時価総額（発行済み株式数ベース）      False
時価総額（普通株式数ベース）        False
東証業種コード３３分類           False
日経業種区分                False
日経業種中分類コード            False
日経業種小分類コード            False
決算月（年２回決算）            False
決算月                   False
発行済み株式数（権利落ベース）       False
発行済み株式数（権利落ベース）変更日    False
日経平均採用区分              False
日経３００採用区分             False
日経５００採用区分             False
東証：上場日                False
東証：上場廃止日              False
dtype: bool

In [27]:
#全ての要素が欠損値となる行が存在するか
# df_tmp = df.drop(columns=["期間", "日経平均", "TOPIX", "東証2部総合", "東証マザーズ", "JASDAQ", "銘柄名称"])
df_tmp = df.drop(columns=df.columns[[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]])
count = 0
is_null_count = 0
for x in df_tmp.isnull().all(axis=1):
    if x == False:
        count += 1
    else:
        is_null_count += 1
print(f"OK_data：", count)
print(f"NaN：{is_null_count}")

OK_data： 0
NaN：379080
